# 학습 관련 기술들

이번 장에서 알아볼 내용
- 최적화 방법
- 가중치 매개변수 초깃값 (전에는 random으로 설정)
- 하이퍼파라미터 설정 방법 (node, lr, batch size....)
- 오버피팅 방지 (가중치 감소, 드랍아웃 등 정규화 방법)
- 배치 정규화

In [2]:
# 라이브러리
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
path = '/content/drive/MyDrive/deep-learning-from-scratch-master'
os.chdir(path)
sys.path.append(os.chdir)

# 매개변수 갱신

매개변수의 최적값을 찾는 이런 과정을 최적화(Optimization)라고 한다.

최적의 매개변수를 찾을 단서로 매개변수의 기울기(1.수치미분 2.오차역전파법)을 앞서 살펴봤음

GD의 수식

$W \leftarrow W - \eta\frac{\partial{L}}{\partial{W}}$

아래와 같이 구현하면 SGD의 인스턴스에 매개변수와 기울기 정보를 전달하면 매개변수가 update 

참고내용 Lasagne 프레임워크의 최적화 기법들 

https://github.com/Lasagne/Lasagne/blob/master/lasagne/updates.py

In [4]:
# Gradient Descent 구현

class SGD(object):
    def __init__(self, lr = 0.01):
        self.lr = lr  # 인스턴스 변수로 learning rate 

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

SGD의 문제점

방향에 따라 기울기가 달라지는 함수에서는 탐색 경로가 비효율적이다.

따라서 개선된 최적화 기법을 살펴보자:
- 모멘텀(Momentum)
- AdaGrad(Adaptive Gradient)
- Adam


In [7]:
# 모멘텀
class Momentum(object):
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr # learning rate 
        self.momentum = momentum # 일반적으로 0.9 (따라서 default값 0.9)
        self.v = None # V(0) = 0

    def update(self, params, grads):
        if self.v is None: # 처음 갱신
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]     

In [9]:
# AdaGrad
class Adagrad(object):
    def __init__(self, lr = 0.01):
        self.lr = lr
        self.h = None

    def update(self, params, grads):
        if self.h is None: # 처음 갱신
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / ((np.sqrt(self.h[key])) + 1e-7)

In [10]:
W1 = np.array([[1, 2], [3, 4]])
W2 = np.array([[2, 4], [6, 8]])
print(W1 * W2) # element wise

[[ 2  8]
 [18 32]]


RMSProp은

Adagrad의 식에서 $h$ 부분이 다름
$h_i \leftarrow ph + (1-p)\frac{\partial{L}}{\partial{W}}\odot\frac{\partial{L}}{\partial{W}}$

$p$ 값이 작을수록 최신 기울기를 많이 반영

In [11]:
# RMSProp
class RMSProp(object):
    def __init__(self, lr = 0.01, p = 0.5):
        self.lr = lr
        self.p = p # 추가
        self.h = None

    def update(self, params, grads):
        if self.h is None: # 처음 갱신
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] = self.p * self.h[key] + (1 - self.p) * (grads[key] * grads[key]) # 식 반영
            params[key] -= self.lr * grads[key] / ((np.sqrt(self.h[key])) + 1e-7)